In [ ]:
%load_ext autoreload
%autoreload 2

# POS and NER tagging of dat

In [1]:

# Project Gutenberg, 244: A Study in Scarlet (en), Arthur Conan Doyle
text_en = """
This was a lofty chamber, lined and littered with countless bottles.
Broad, low tables were scattered about, which bristled with retorts,
test-tubes, and little Bunsen lamps, with their blue flickering flames.
There was only one student in the room, who was bending over a distant
table absorbed in his work. At the sound of our steps he glanced round
and sprang to his feet with a cry of pleasure. “I’ve found it! I’ve
found it,” he shouted to my companion, running towards us with a
test-tube in his hand. “I have found a re-agent which is precipitated
by hæmoglobin, and by nothing else.” Had he discovered a gold mine,
greater delight could not have shone upon his features.
""".strip()

In [2]:
import nltk

nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\roger\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [3]:
headers = ['term', 'nltk (pos)', 'nltk (universal)', 'spaCy', 'LLM']
rows = []

tokens = nltk.word_tokenize(text_en)



for token in nltk.word_tokenize(text_en):
  if not token.isalpha():
    continue
  row = [token]
  row.append(nltk.pos_tag([token]))
  row.append(nltk.pos_tag([token], tagset='universal'))
  rows.append(row)
  if len(rows) > 30:
    break



LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - 'C:\\Users\\roger/nltk_data'
    - 'c:\\env\\python310\\nltk_data'
    - 'c:\\env\\python310\\share\\nltk_data'
    - 'c:\\env\\python310\\lib\\nltk_data'
    - 'C:\\Users\\roger\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
